In [1]:
# Import to make brainrender embed scenes in jupyter notebooks
from brainrender.scene import Scene
from brainrender import __version__, settings
from rich import print
from vedo import embedWindow

from myterial import blue_light, salmon

settings.SHOW_AXES = False
settings.WHOLE_SCREEN = False

embedWindow(None)
if not __version__=='2.0.2.9':
    raise ValueError(f'This executable is meant to work with brainrender 2.0.2.9, not: {__version__}')

def make_scene(species='mouse'):
    if species == 'mouse':
        scene = Scene()
    elif species == 'zfish':
        scene = Scene(atlas_name="mpin_zfish_1um",)
    else:
        raise ValueError(f'Species not supported: {species}')
    scene.root._needs_silhouette = True
    scene.root._silhouette_kwargs['lw'] = 1
    return scene
    
def render_scene(scene, **kwargs):
    print(f'[{blue_light}]Rendering scene, press "[{salmon} b]q[/{salmon} b]" to close')
    scene.render(**kwargs)

    # close when done
    scene.plotter.close()

# Introduction

Understanding how nervous systems generate behavior benefits from gathering multidimensional data from different individual animals. These data range from neural activity recordings and anatomical connectivity, to cellular and subcellular information such as morphology and gene expression profiles. These different types of data should ideally all be in register so that, for example, neural activity in one brain region can be interpreted in light of the connectivity of that region or the cell types it contains. Such registration, however, is challenging. Often it is not technically feasible to obtain multidimensional data in a single experiment, and registration to a common reference frame must be performed post hoc. Even for the same experiment type, registration is necessary to allow comparisons across individual animals [@bib25].

While different types of references can in principle be used, neuroanatomical location is a natural and most commonly used reference frame [@bib6; @bib21; @bib2; @bib15]. In recent years, several high-resolution three-dimensional (3D) digital brain atlases have been generated for model species commonly used in neuroscience [@bib32; @bib21; @bib2; @bib15]. These atlases provide a framework for registering different types of data across macro- and microscopic scales. A key output of this process is the visualization of all datasets in register. Given the intrinsically 3D geometry of brain structures and individual neurons, 3D renderings are more readily understandable and can provide more information when compared to two dimensional images. Exploring interactive 3D visualizations of the brain gives an overview of the relationship between datasets and brain regions and helps generating intuitive insights about these relationships. This is particularly important for large-scale datasets such as the ones generated by open-science projects like MouseLight [@bib33] and the Allen Mouse Connectome [@bib21]. In addition, high-quality 3D visualizations facilitate the communication of experimental results registered to brain anatomy.

Generating custom 3D visualizations of atlas data requires programmatic access to the atlas. While some of the recently developed atlases provide an API (Application Programming Interface) for accessing atlas data [@bib32; @bib15], rendering these data in 3D remains a demanding and time-consuming task that requires significant programming skills. Moreover, visualization of user-generated data registered onto the atlas requires an interface between the user data and the atlas data, which further requires advanced programming knowledge and extensive development. There is therefore the need for software that can simplify the process of visualizing 3D anatomical data from available atlases and from new experimental datasets.

Currently, existing software packages such as cocoframer [@bib16], BrainMesh [@bib34], and SHARPTRACK [@bib24] provide some functionality for 3D rendering of anatomical data. These packages, however, are only compatible with a single atlas and cannot be used to render data from different atlases or different animal species. Achieving this requires adapting the existing software to the different atlases datasets or developing new dedicated software all together, at the cost of significant additional efforts, often duplicated. An important limitation of the currently available software is that it frequently does not support rendering of non-atlas data, such as data from publicly available datasets (e.g. MouseLight) or produced by individual laboratories. This capability is essential for easily mapping newly generated data onto brain anatomy at high resolution and produce visualizations of multidimensional datasets. More advanced software such as natverse [@bib5] offers extensive data visualization and analysis functionality, but currently, it is mostly restricted to data obtained from the _Drosophila_ brain. Simple Neurite Tracer [@bib3], an ImageJ-based software, can render neuronal morphological data from public and user-generated datasets and is compatible with several reference atlases. However, this software does not support visualization of data other than neuronal morphological reconstructions nor can it be easily adapted to work with different or new atlases beyond the ones already supported. Finally, software such as MagellanMapper [@bib35] can be used to visualize and analyze large 3D brain imaging datasets, but the visualization is restricted to one data item (i.e. images from one individual brain). It is therefore not possible to combine data from different sources into a single visualization. Ideally, a rendering software should work with 3D mesh data instead of 3D voxel image data to allow the creation of high-quality renderings and facilitate the integration of data from different sources.

An additional consideration is that existing software tools for programmatic neuroanatomical renderings have been developed in programming languages such as R and Matlab, and there is currently no available alternative in Python. The popularity of Python within the neuroscientific community has grown tremendously in recent years [@bib19]. Building on Python’s simple syntax and free, high-quality data processing and analysis packages, several open-source tools directly aimed at neuroscientists have been written in Python and are increasingly used (e.g., [@bib18]; [@bib22]; [@bib31]). Developing a python-based software for universal generation of 3D renderings of anatomically registered data can therefore take advantage of the increasing strength and depth of the python neuroscience community for testing and further development.

For these reasons, we have developed brainrender: an open-source python package for creating high-resolution, interactive 3D renderings of anatomically registered data. Brainrender is written in Python and integrated with BrainGlobe’s AtlasAPI [@bib7] to interface natively with different atlases without need for modification. Brainrender supports the visualization of data acquired with different techniques and at different scales. Data from multiple sources can be combined in a single rendering to produce rich and informative visualizations of multidimensional data. Brainrender can also be used to create high-resolution, publication-ready images and videos (see [@bib31]; [@bib1]), as well as interactive online visualizations to facilitate the dissemination of anatomically registered data. Finally, using brainrender requires minimal programming skills, which should accelerate the adoption of this new software by the research community. All brainrender code is available at the GitHub repository together with extensive online documentation and examples.

# Results

## Design principles and implementation

A core design goal for brainrender was to generate a visualization software compatible with any reference atlas, thus providing a generic and flexible tool ([Figure 1A](#fig1)). To achieve this goal, brainrender has been developed as part of the BrainGlobe’s computational neuroanatomy software suite. In particular, we integrated brainrender directly with BrainGlobe’s AtlasAPI [@bib7]. The AtlasAPI can download and access atlas data from several supported atlases in an unified format. Brainrender uses the AtlasAPI to access 3D mesh data from individual brain regions as well as metadata about the hierarchical organization of the brain’s structures ([Figure 1B](#fig1)). Thus, the same programming interface can be used to access data from any atlas (see code examples in [Figure 2](#fig2)), including recently developed ones (e.g. the enhanced and unified mouse brain atlas, [@bib6]).

figure: Figure 1.
:::
![](article.ipynb.media/fig1.jpg)

### Design principles.

(**A**) Schematic illustration of how different types of data can be loaded into brainrender using either brainrender’s own functions, software packages from the BrainGlobe suite, or custom Python scripts. All data loaded into brainrender is converted to a unified format, which simplifies the process of visualizing data from different sources. (**B**) Using brainrender with different atlases. Visualization of brain atlas data from three different atlases using brainrender. Left, Allen atlas of the mouse brain showing the superficial (SCs) and motor (SCm) subdivisions of the superior colliculus and the Zona Incerta (data from [@bib32]). Middle, visualization of the cerebellum and tectum in the larval zebrafish brain (data from [@bib15]). Right, visualization of the precentral gyrus, postcentral gyrus, and temporal lobe of the human brain (data from [@bib8]). (**C**) The brainrender GUI. Mouse, human, and zebrafish larvae drawings from [scidraw.io](https://scidraw.io/) ([doi.org/10.5281/zenodo.3925991](http://doi.org/10.5281/zenodo.3925991), [doi.org/10.5281/zenodo.3926189](http://doi.org/10.5281/zenodo.3926189), [doi.org/10.5281/zenodo.3926123](http://doi.org/10.5281/zenodo.3926123)).
:::
{#fig1}


figure: Figure 2.
:::
![](article.ipynb.media/fig2.jpg)

### Code examples.

Example python code for visualizing brain regions in the mouse and larval zebrafish brains. The same commands can be used for both atlases and switching between atlases can be done by simply specifying which atlas to use when creating the visualization. Further examples can be found in brainrender’s GitHub repository.
:::
{#fig2}




The second major design principle was to enable rendering of any data type that can be registered to a reference atlas, either from publicly available datasets or from individual laboratories. Brainrender can directly visualize data produced with any analysis software from the BrainGlobe suite, including cellfinder [@bib30] and brainreg [@bib31]. In addition, brainrender provides functionality for easily loading and visualizing commonly used data types such as .npy files with cell coordinates or image data, .obj, and .stl files with 3D mesh data and .json files with streamlines data for mesoscale connectomics. Additional information about the file formats accepted by brainrender can be found in the online documentation. Brainglobe’s software suite also includes imio which can load data from several file types (e.g. tiff and .nii), and additional file formats can be loaded through the numerous packages provided by the python ecosystem. Finally, the existing loading functionality can be easily expanded to support user-specific needs by directly plugging in custom user code into the brainrender interface ([Figure 1A](#fig1)).

One of the goals of brainrender is to facilitate the creation of high-resolution images, animated videos, and interactive online visualizations from any anatomically registered data. Brainrender uses vedo as the rendering engine [@bib20], a state-of-the-art tool that enables fast, high-quality rendering with minimal hardware requirements.

High-resolution renderings of rich 3D scenes can be produced rapidly (e.g. 10,000 cells in less than 2 s) in standard laptop or desktop configurations. Benchmarking tests across different operating systems and machine configurations show that using a GPU can increase the framerate of interactive renderings by a factor of 3.5 (see [Tables 1](#table1) and [2](#table2) in Materials and methods). This performance increase, however, depends on the complexity of the pre-processing steps, such as data loading and mesh generation, which run on the CPU. As one the main goals of brainrender is to produce high-resolution visualizations, we have made the rendering quality independent of hardware configuration, which only affects the rendering time. Animated videos and online visualizations can be produced with a few lines of code in brainrender. Several options are provided for easily customizing the appearance of rendered objects, thus enabling high-quality, rich data visualizations that combine multiple data sources.

table: Table 1.
:::
### Machine configurations used for benchmark tests.

| N | OS                        | CPU                                     | GPU                        |
| - | ------------------------- | --------------------------------------- | -------------------------- |
| 1 | Macos Mojave 10.14.6      | 2.3 ghz Intel Core i9                   | Radeon Pro 560 × 4 GB GPU  |
| 2 | Ubuntu 18.04.2 LTS x86 64 | Intel i7-8565U (x) @ 4.5 ghz            | NO GPU                     |
| 3 | Windows 10                | Intel(R) Core i7-7700HQ 2.8 ghz         | NO GPU                     |
| 4 | Windows 10                | Intel(R) Xeon(R) CPU E5-2643 v3 3.4 ghz | NVIDIA geforce GTX 1080 Ti |
:::
{#table1}

table: Table 2.
:::
### Benchmark tests results.

The number of actors refers to the total number of elements rendered, and the number of vertices refers to the total number of mesh vertices in the rendering.

| Test                | Machine | GPU | # actors | # vertices | FPS    | Run duration |
| ------------------- | ------- | --- | -------- | ---------- | ------ | ------------ |
| 10 k cells          | 1       | Yes | 3        | 1,029,324  | 24.76  | 0.81         |
|                     | 2       | No  | 3        | 1,029,324  | 22.46  | 1.16         |
|                     | 3       | No  | 3        | 1,029,324  | 20.00  | 1.41         |
|                     | 4       | Yes | 3        | 1,029,324  | 100.00 | 1.34         |
| 100 k cells         | 1       | Yes | 3        | 9,849,324  | 18.87  | 3.23         |
|                     | 2       | No  | 3        | 9,849,324  | 14.91  | 4.34         |
|                     | 3       | No  | 3        | 9,849,324  | 0.43   | 7.94         |
|                     | 4       | Yes | 3        | 9,849,324  | 1.20   | 1.13         |
| 1 M cells           | 1       | Yes | 3        | 98,049,324 | 2.65   | 31.01        |
|                     | 2       | No  | 3        | 98,049,324 | 2.55   | 96.49        |
|                     | 3       | No  | 3        | 98,049,324 | 0.03   | 86.75        |
|                     | 4       | Yes | 3        | 9,8049,324 | 0.13   | 36.57        |
| Slicing 10 k cells  | 1       | Yes | 3        | 237,751    | 37.64  | 0.96         |
|                     | 2       | No  | 3        | 237,751    | 39.10  | 1.25         |
|                     | 3       | No  | 3        | 237,751    | 26.32  | 1.88         |
|                     | 4       | Yes | 3        | 237,751    | 200.00 | 1.34         |
| Slicing 100 k cells | 1       | Yes | 3        | 276,092    | 31.79  | 7.77         |
|                     | 2       | No  | 3        | 276,092    | 25.98  | 9.09         |
|                     | 3       | No  | 3        | 276,092    | 21.28  | 16.88        |
|                     | 4       | Yes | 3        | 276,092    | 111.11 | 9.65         |
| Slicing 1 M cells   | 1       | Yes | 3        | 275,069    | 11.23  | 91.31        |
|                     | 2       | No  | 3        | 275,069    | 5.39   | 104.79       |
|                     | 3       | No  | 3        | 275,069    | 5.03   | 158.99       |
|                     | 4       | Yes | 3        | 275,069    | 37.04  | 97.43        |
| Brain regions       | 1       | Yes | 1678     | 1,864,388  | 9.38   | 11.78        |
|                     | 2       | No  | 1678     | 1,864,388  | 7.61   | 27.40        |
|                     | 3       | No  | 1678     | 1,864,388  | 6.49   | 46.79        |
|                     | 4       | Yes | 1678     | 1,864,388  | 11.90  | 35.83        |
| Animation           | 1       | Yes | 8        | 96,615     | 9.91   | 18.98        |
|                     | 2       | No  | 8        | 96,615     | 22.12  | 12.63        |
|                     | 3       | No  | 8        | 96,615     | 15.15  | 11.92        |
|                     | 4       | Yes | 8        | 96,615     | 47.62  | 12.29        |
| Volume              | 1       | Yes | 12       | 49,324     | 1.79   | 2.31         |
|                     | 2       | No  | 12       | 49,324     | 1.66   | 1.95         |
|                     | 3       | No  | 12       | 49,324     | 3.55   | 2.15         |
|                     | 4       | Yes | 12       | 49,324     | 23.26  | 1.21         |
:::
{#table2}

Finally, we aimed for brainrender to empower scientists with little or no programming experience to generate advanced visualizations of their anatomically registered data. To make brainrender as user-friendly as possible we have produced extensive documentation, tutorials and examples for installing and using the software. We have also developed a graphic user interface (GUI) to access most of brainrender’s core functionality. This GUI can be used to perform actions such as rendering of brain regions and labeled cells (e.g. from cellfinder) and creating images of the rendered data, without writing custom python code ([Figure 1C](#fig1)), ([Video 1](#video1)).



## Visualizing brain regions and other structures

A key element of any neuroanatomical visualization is the rendering of the entire outline of the brain as well as the borders of brain regions of interest. In brainrender, this can easily be achieved by specifying which brain regions to include in the rendering. The software will then use BrainGlobe’s AtlasAPI to load the 3D data and subsequently renders them ([Figure 1B](#fig1)).

Brainrender can also render brain areas defined by factors other than anatomical location, such as gene expression levels or functional properties. These can be loaded either directly as 3D mesh data after processing with dedicated software (e.g., [@bib30]; [@bib27]; [@bib13]; [Figure 3A](#fig3)) or as 3D volumetric data ([Figure 3E](#fig3)). For the latter, brainrender takes care of the conversion of voxels into a 3D mesh for rendering. Furthermore, custom 3D meshes can be created to visualize different types of data. For example, brainrender can import JSON files with tractography connectivity data and create ‘streamlines’ to visualize efferent projections from a brain region of interested ([Figure 3B](#fig3)).


In [2]:
# Figure 3
'''

figure: Figure 3.
:::
![](article.ipynb.media/fig3.jpg)

### Visualizing different types of data in brainrender.

(**A**) Spread of fluorescence labeling following viral injection of AAV2-CRE-eGPF in the superior colliculus of two FLEX-TdTomato mice. 3D objects showing the injection sites were created using custom python scripts following acquisition of a 3D image of the entire brain with serial two-photon tomography and registration of the image data to the atlas’ template (with brainreg, [@bib30]). (**B**) Streamlines visualization of efferent projections from the mouse primary motor cortex following injection of an anterogradely transported virus expressing fluorescent proteins (original data from [@bib21]), downloaded from (Neuroinformatics NL with brainrender). (**C**) Visualization of the location of several implanted neuropixel probes from multiple mice (data from [@bib28]). Dark salmon colored tracks show probes going through both primary/anterior visual cortex (VISp/VISa) and the dorsal lateral geniculate nucleus of the thalamus. (**D**) Single periaqueductal gray (PAG) neuron. The PAG and superior colliculus are also shown. The neuron’s morphology was reconstructed by targeting the expression of fluorescent proteins in excitatory neurons in the PAG via an intersectional viral strategy, followed by imaging of cleared tissue and manual reconstruction of the neuron’s morphology with Vaa3D software. Data were registered to the Allen atlas with SHARPTRACK [@bib24]. The 3D data was saved as a .stl file and loaded directly into brainrender. (**E**) Gene expression data. Left, expression of genes ‘brn3c’ and ‘nk1688CGt’ in the tectum of the larval zebrafish brain (gene expression data from fishatlas.neuro.mpg.de, 3D objects created with custom python scripts). Right, expression of gene ‘Gpr161’ in the mouse hippocampus (gene expression data from [@bib32]), downloaded with brainrender (3D objects created with brainrender). Colored voxels show voxels with high gene expressions. The CA1 field of the hippocampus is also shown.
:::
{#fig3}
'''
from myterial import indigo as scmcol
from myterial import indigo_dark as scscol
from myterial import blue_darker as zicol



# Panel (A)
scene = make_scene()
for reg, col in zip(("SCm", "SCs", "ZI"), (scmcol, scscol, zicol)):
    scene.add_brain_region(reg, color=col, silhouette=True)
render_scene(scene)


Rendering scene, press " q " to close

In [3]:
# Panel (B)
from myterial import blue_grey
from myterial import salmon_dark as streamlinescol

from brainrender.atlas_specific import get_streamlines_for_region
from brainrender.actors.streamlines import Streamlines

scene = make_scene()

# get streamlines data
streams = get_streamlines_for_region("MOp")

# add Streamlines actors
s = scene.add(Streamlines(streams[0], color=streamlinescol, alpha=1))

# add brain regions
th = scene.add_brain_region(
    "TH", alpha=0.45, silhouette=False, color=blue_grey
)


render_scene(scene)

downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

Rendering scene, press " q " to close

In [4]:
# Panel (C)
from brainrender.actors import Points

try:
    from oneibl.onelight import ONE
except ImportError:
    %pip install ibllib
    from oneibl.onelight import ONE

from myterial import blue_grey, blue_grey_dark, salmon_light, salmon_darker

scene = make_scene()


# download probe data from ONE
one = ONE()
one.set_figshare_url("https://figshare.com/articles/steinmetz/9974357")

# select sessions with trials
sessions = one.search(["trials"])

# get probe locations
probes_locs = []
for sess in sessions:
    probes_locs.append(one.load_dataset(sess, "channels.brainLocation"))

# get single probe tracks
for locs in probes_locs:
    k = int(len(locs) / 374.0)

    for i in range(k):
        points = locs[i * 374 : (i + 1) * 374]
        regs = points.allen_ontology.values

        # color based on if probes go through selected regions
        if "LGd" in regs and ("VISa" in regs or "VISp" in regs):
            color = salmon_darker
            alpha = 1
            sil = 1
        elif "VISa" in regs:
            color = salmon_light
            alpha = 1
            sil = 0.5
        else:
            continue

        # render channels as points
        spheres = Points(
            points[["ccf_ap", "ccf_dv", "ccf_lr"]].values,
            colors=color,
            alpha=alpha,
            radius=30,
        )
        spheres = scene.add(spheres)
        scene.add_silhouette(spheres, lw=sil)


# Add brain regions
visp, lgd = scene.add_brain_region(
    "VISp",
    "LGd",
    hemisphere="right",
    alpha=0.3,
    silhouette=False,
    color=blue_grey_dark,
)
visa = scene.add_brain_region(
    "VISa",
    hemisphere="right",
    alpha=0.2,
    silhouette=False,
    color=blue_grey,
)
th = scene.add_brain_region(
    "TH", alpha=0.3, silhouette=False, color=blue_grey_dark
)
th.wireframe()
scene.add_silhouette(lgd, visp, lw=2)

render_scene(scene)

Rendering scene, press " q " to close

In [16]:
# Panel (D)
from brainrender.actors import Point

from myterial import blue_grey_light as scmcol
from myterial import blue_grey as pagcol
from myterial import salmon_dark as neuroncol

cam = {
    "pos": (-16954, 2456, -3961),
    "viewup": (0, -1, 0),
    "clippingRange": (22401, 34813),
    "focalPoint": (7265, 2199, -5258),
    "distance": 24256,
}

scene = make_scene()

# add brain regions
pag = scene.add_brain_region("PAG", alpha=0.4, silhouette=False, color=pagcol)
scm = scene.add_brain_region("SCm", alpha=0.3, silhouette=False, color=scmcol)

# add neuron mesh
neuron = scene.add(r"C:\Users\Federico\Documents\GitHub\BrainRender\paper\data\yulins_neuron.stl")
neuron.c(neuroncol)

# add sphere at soma location
soma_pos = [9350.51912036, 2344.33986638, 5311.18297796]
point = scene.add(Point(soma_pos, color=neuroncol, radius=25))
scene.add_silhouette(point, lw=1, color="k")
scene.add_silhouette(neuron, lw=1, color="k")

# slice scene repeatedly to cut out region of interest
p = [9700, 1, 800]
plane = scene.atlas.get_plane(pos=p, plane="frontal")
scene.slice(plane, actors=[scm, pag, scene.root])

p = [11010, 5000, 5705]
plane = scene.atlas.get_plane(pos=p, norm=[0, -1, 0])
scene.slice(plane, actors=[scene.root])

# render
render_scene(scene, camera=cam, zoom=9)


Rendering scene, press " q " to close

In [5]:
# Panel (E)
from brainrender.actors import Volume
from myterial import purple_dark as gene2_color
from myterial import purple_light as gene1_color

cam = cam = {
    "pos": (-835, -1346, 1479),
    "viewup": (0, -1, 0),
    "clippingRange": (1703, 3984),
    "focalPoint": (334, 200, -342),
    "distance": 2660,
}


# shift parameters to adjust meshes position
SHIFT = [-20, 15, 30]  # fine tune pos

scene = make_scene(species='zfish')

# add custom meshes from file, but don't show them (alpha=0)
m = scene.add(r"C:\Users\Federico\Documents\GitHub\BrainRender\paper\data\T_AVG_brn3c_GFP.obj", color=gene1_color, alpha=0)
m2 = scene.add(r"C:\Users\Federico\Documents\GitHub\BrainRender\paper\data\T_AVG_nk1688CGt_GFP.obj", color=gene2_color, alpha=0)

# convert meshes to volumetric data showing gene expression density
vol1 = Volume(m.mesh.density(), as_surface=True, min_value=20000, cmap="Reds")
vol1.lw(1)
scene.add(vol1)

vol2 = Volume(m2._mesh.density(), as_surface=True, min_value=600, cmap="Blues")
vol2.lw(1)
scene.add(vol2)

render_scene(scene, zoom=2.5, camera=cam)

FileNotFoundError: File C:\Users\Federico\Documents\GitHub\BrainRender\paper\data\T_AVG_brn3c_GFP.obj not found



Brainrender also simplifies visualizing the location of devices implanted in the brain for neural activity recordings or manipulations, such as electrodes or optical fibers. Post hoc histological images taken to confirm the correct placement of the device can be registered to a reference atlas using appropriate software, and the registered data can be imported into brainrender ([Figure 3C](#fig3)). This type of visualization greatly facilitates cross-animal comparisons and helps data interpretation within and across research groups.

Finally, brainrender can be used to visualize any object represented by the most commonly used file formats for 3D design (e.g. .obj, .stl), thus ensuring that brainrender can flexibly adapt to the visualization needs of the user ([Figure 3D](#fig3)).

## Individual neurons and mesoscale connectomics

Recent advances in large field of view and whole-brain imaging allow the generation of brain-wide data at single neuron resolution. Having a platform for visualizing these datasets with ease is critical for exploratory data analyses. Several open-source software packages are available for registering large amounts of such imaging data and automatically identify labeled cells (e.g. expressing fluorescent proteins) [@bib30; @bib9; @bib10; @bib23]. This processing step outputs a table of coordinates for a set of labeled cells, which can be directly imported into brainrender to visualize a wealth of anatomical data at cellular resolution ([Figure 4A](#fig4)).

In [ ]:
# Figure 4

'''

figure: Figure 4.
:::
![](article.ipynb.media/fig4.jpg)

### Visualizing cell location and morphological data.

(**A**) Visualizing the location of labeled cells. Left, visualization of fluorescently labeled cells identified using cellfinder (data from [@bib31]). Right, visualization of functionally defined clusters of regions of interest in the brain of a zebrafish larvae during a visuomotor task (data from [@bib17]). (**B**) Visualizing neuronal morphology data. Left, three secondary motor cortex neurons projecting to the thalamus (data from [@bib33], downloaded with morphapi from [neuromorpho.org](http://neuromorpho.org/), [@bib4]). Right, morphology of cerebellar neurons in larval zebrafish (data from [@bib15]), (downloaded with morphapi). In the left panel of (**A** and **B**), the brain outline was sliced along the midline to expose the data.
:::
{#fig4}
'''



Beyond the location of cell bodies, visualizing the entire dendritic and axonal arbors of single neurons registered to a reference atlas is important for understanding the distribution of neuronal signals across the brain. Single-cell morphologies are often complex 3D structures and therefore poorly represented in 2D images. Generating 3D interactive renderings is thus important to facilitate the exploration of this type of data. Brainrender can be used to parse and render .swc files containing morphological data, and it is fully integrated with morphapi, a software for downloading morphological data from publicly available datasets (e.g. from [neuromorpho.org](http://neuromorpho.org/), [@bib4]; [Figure 4B](#fig4)).

## Producing figures, videos, and interactive visualizations with brainrender

A core goal of brainrender is to facilitate the production of high-quality images, videos, and interactive visualizations of anatomical data. Brainrender leverages the functionality provided by vedo [@bib20] to create images directly from the rendered scene. Renderings can also be exported to HTML files to create interactive visualizations that can be hosted online. Finally, functionality is provided to easily export videos from rendered scenes. Animated videos can be created by specifying parameters (e.g. the position of the camera or the transparency of a mesh) at selected keyframes. Brainrender then creates a video by animating the rendering between the keyframes. This approach facilitates the creation of videos while retaining the flexibility necessary to produce richly animated sequences ([Videos 2](#video2)–[5](#video5)). All example figures and videos in this article were generated directly in brainrender, with no further editing.



# Discussion

In this article, we have presented brainrender, a python software for creating 3D renderings of anatomically registered data.

Brainrender addresses the current lack of python-based and user-friendly tools for redeanatomical data. Being part with BrainGlobe’s suite of software tools for the analysis of anatomical data brainrender facilitates the development of integrated analysis pipelines and the re-usability of software tools across model species, minimizing the need for additional software development. Finally, brainrender promises to improve how anatomically registered data are disseminated both in scientific publications and in other media (e.g., hosted online).

## Limitations and future directions

With brainrender, we aimed to make the rendering process as simple as possible. Nevertheless, some more technically demanding pre-processing steps of raw image data are necessary before they can be visualized in brainrender. In particular, a critical step for visualizing anatomical data is the registration to a reference template (e.g., one of the atlases provided by the AtlasAPI). While this step can be challenging and time-consuming, the brainglobe suite provides software to facilitate this process (e.g., brainreg and bg-space), and alternative software tools have been developed before for this purpose (e.g., [@bib27]; [@bib13]). Additional information about data registration can be found in brainglobe’s and brainrender’s online documentation, as well as in the examples in brainrender’s GitHub repository. A related challenge is integrating new anatomical atlases into the AtlasAPI. While we anticipate that most users will not have this need, it is a non-trivial task that requires considerable programming skills. We believe that brainglobe’s AtlasAPI greatly facilitates this process, which is presented in [@bib7] and has extensive online documentation.

Brainrender has been optimized for rendering quality instead of rendering performance. Other commonly used software tools like napari [@bib26] and ImageJ are dedicated to visualizing N-dimensional image data and perform very well even on large datasets. When comparing brainrender with other software, it is important to note brainrender is intended to work primarily with mesh data and not 3D image data. Although it can display image data (e.g., with the Volume actor), this functionality is not as fully developed as that using mesh data. A direct benchmarking comparison between brainrender and napari shows that brainrender is 5× slower than napari at visualizing image data, but 20× faster at visualizing mesh data. In both cases, however, brainrender achieves superior rendering quality. Other software packages dedicated to high-performance rendering, such as Blender, can handle mesh data with a performance that surpasses brainrender. Their use, however, comes with the large overhead of learning a very complex software to generate what most often will be simple renderings. It also requires that the users themselves take care of downloading, storing, and accessing mesh data from the anatomical atlases. Nevertheless, the rendering performance of brainrender could be a target for improvement in future versions, both for images and for mesh data, through optimizing the Actor classes. While we have designed brainrender usage to require minimal programming expertise, installing python and brainrender may still prove challenging for some users. In the future, we aim to make brainrender a stand-alone application that can be simply downloaded and locally installed, either through Docker containers or through executable files. Further possible improvements include the development of plug-ins for loading of data from file formats other than those already supported, and improvements to the GUI functionality. Moreover, in addition to images and videos, brainrender can be used to export renderings as HTML files and generate online 3D interactive renderings. Currently, however, embedding renderings into a web page remains far from a trivial task. Further developments on this front should make it possible to easily host interactive renderings online, therefore improving how anatomically registered data are disseminated both in scientific publications and in other media. While we plan to continue developing brainrender in the future, we welcome contributions from the community. Users should feel encouraged to contribute irrespective of their programming experience, and we note that the programming ability of many biologists is often better than what they perceive it to be. We especially welcome contributions aimed at improving the user-experience of brainrender, at any level of interaction. Contributions can involve active development of brainrender’s code base, but they can also be bug reports, features request, improvements with the online documentation, and help answering users’ questions.

# Materials and methods

table: Key resources table
:::
| Reagent type (species) or resource | Designation          | Source or reference                                                              | Identifiers                                                       | Additional information |
| ---------------------------------- | -------------------- | -------------------------------------------------------------------------------- | ----------------------------------------------------------------- | ---------------------- |
| Software, algorithm                | Numpy                | <https://doi.org/10.1038/s41586-020-2649-2>                                      | RRID:[SCR_008633](https://scicrunch.org/resolver/RRID:SCR_008633) |                        |
| Software, algorithm                | Vtk                  | <https://doi.org/10.1016/j.softx.2015.04.001>                                    | RRID:[SCR_015013](https://scicrunch.org/resolver/RRID:SCR_015013) |                        |
| Software, algorithm                | Vedo                 | <https://zenodo.org/record/4287635>                                              |                                                                   |                        |
| Software, algorithm                | BrainGlobe Atlas API | <https://doi.org/10.21105/joss.02668>                                            |                                                                   |                        |
| Software, algorithm                | Pandas               | <https://doi.org/10.5281/zenodo.3509134>                                         |                                                                   |                        |
| Software, algorithm                | Matplotlib           | doi: [10.1109/MCSE.2007.55](https://doi.org/10.1109/MCSE.2007.55)                | RRID:[SCR_008624](https://scicrunch.org/resolver/RRID:SCR_008624) |                        |
| Software, algorithm                | Jupyter              | doi:[10.3233/978-1-61499-649-1-87](https://doi.org/10.3233/978-1-61499-649-1-87) | RRID:[SCR_018416](https://scicrunch.org/resolver/RRID:SCR_018416) |                        |
:::
{#keyresource}

## Brainrender’s workflow

Brainrender is written in Python three and depends on standard python packages such as numpy, matplotlib, and pandas [@bib11; @bib12; @bib29] and on vedo [@bib20] and BrainGlobe’s AtlasAPI [@bib7]. Extensive documentation on how to install and use brainrender can be found at docs.brainrender.info, and we provide here a only brief overview of the workflow in brainrender. The GitHub repository also contains detailed examples of Python scripts and Jupyter notebooks [@bib14]. All brainrender’s code is open-source and has been deposited in full in the GitHub repository and at PyPI (a repository of Python software) under a permissive BSD 3-Clause license. We welcome any user to download and inspect the source code, modify it as needed, or contribute to brainrender’s development directly.

Brainrener can be installed in any python environment using python version ≥ 3.60.0. We recommend the creation of an anaconda or virtual environment with an appropriate python version for use with brainrender. Installing brainrender is then as simple as ‘pip install brainrender’ although additional optional packages might have to be installed separately (e.g., to access data from the Allen Institute).

The central element of any visualization produced by brainrender is the Scene. A Scene controls which elements (Actors) are visualized and coordinates the rendering, the position of the camera’s point of view, the generation of screenshots and animations from the rendered scene, and other important actions.

Actors can be added to the scene in several ways. When loading data directly from a file with 3D mesh information (e.g. .obj), an Actor is generated automatically to represent the mesh in the rendering. When rendering data from other sources (e.g. from a .swc file with neuronal morphology or from a table of coordinates of labeled cells), dedicated functions in brainrender parse the input data and generate the corresponding Actors. Actors in brainrender have properties, such as color and transparency, that can be used to specify the appearance of a rendered actor accordingly to the user’s aesthetic preferences. Brainrender’s Scene and Actor functionality use vedo as the rendering engine (GitHub repository; [@bib20]).

In addition to data loaded from external files, brainrender can directly load atlas data containing, for example, the 3D meshes of individual brain regions. This is done via BrainGlobe’s AtlasAPI to allow the same programming interface in brainrender to visualize data from any atlas supported by the AtlasAPI. Brainrender also provides additional functionality to interface with data available from projects that are part of the Allen Institute Mouse Atlas and Mouse Connectome projects [@bib32; @bib21]. These projects provide an SDK (Software Development Kit) to directly download data from their database, and brainrender provides a simple interface for downloading gene expression and connectomics (streamlines) data. All atlas and connectomics data downloaded by brainrender can be loaded directly into a Scene as Actors.

Visualizing morphological data with reconstructions of individual neurons can be done by loading these type of data directly from .swc files or by downloading them in Python using morphapi – software from the BrainGlobe suite that provides a simple and unified interface with several databases of neuron morphologies (e.g., [neuromorpho.org](http://neuromorpho.org/), [@bib4]). Data downloaded with morphapi can be loaded directly into a brainrender scene for visualization.

## Example code

As a demonstration of how easily renderings can be created in brainrender, the Python code ([Figure 5](#fig5)) illustrates how to create a Scene and add Actors by loading 3D data from an .obj file and then adding brain regions to the visualization. Brainrender’s GitHub repository provides several simple and concise examples about how to use brainrender to load user data, atlas data, to edit rendered meshes (e.g., to change color or cut them with a plane), to save screenshots from rendered scenes, and to create animated videos.

figure: Figure 5.
:::
![](article.ipynb.media/fig5.jpg)

### Code examples.

(**A**) Example code to visualize a set of labeled cells coordinates using the Points actor class. (**B**) Code example illustrating how to override brainrender’s default settings and how to use custom camera settings. (**C**) Code example showing how custom mesh objects saved as .obj and .stl files can be visualized in brainrender. (**D**) Example usage of brainrender’s Animation class to create custom animations. Further examples can be found in brainrender’s GitHub repository.
:::
{#fig5}

While brainrender is intended to be mainly a visualization tool, simple analyses can be carried out directly by leveraging functionality from either vedo or BrainGlobe’s AtlasAPI. For example, Vedo can access properties of actors added to a brainrender scene, which could be used to measure the distance between two actors or to check if two actors’ meshes intersect ([Figure 6A](#fig6)). Similarly, BrainGlobe’s AtlasAPI provides methods to, for example, check whether a point (defined by a set of coordinates) is contained in a brain region of interest or to retrieve brain regions that are above or below a brain region of interest in the atlas’ hierarchy ([Figure 6B](#fig6)).

figure: Figure 6.
:::
![](article.ipynb.media/fig6.jpg)

### Advanced code examples.

(**A**) Example code to measure the distance between actors and if a given actor is contained in a target brain region. Left: virus injection volumes (red and gray) reconstructed from virus injections targeted at the superior colliculus (magenta). Gray colored injection volumes show data from the Allen Mouse Connectome [@bib21]. Right: example code to measure the distance between the center of two brainrender actors and to check if an actor’s center is contained in a brain region of interest. (**B**) Code example illustrating how check if a point (e.g., representing a labeled cell) is in a brain region of interest. Left: visualization of reconstructed probe positions from several individual animals, data from [@bib28]. Probe channels located in the thalamus (red) are highlighted. Right: example code showing how to use BrainGlobe’s AtlasAPI to verify whether a point (here representing a probe channel) is contained in a brain region of interest or any of its substructures. Further examples can be found in brainrender’s GitHub repository.
:::
{#fig6}

The code and data used to generate the figures and videos in this article are made freely available at brainrender’s GitHub repository and provides examples of more advanced usage of brainrender’s functionality.

## Benchmark tests

We designed a series of benchmark tests aimed at evaluating brainrender’s performance with different combinations of hardware and operating system. We used five tests designed to cover most aspects of brainrender’s functionality:

-   rendering large numbers (1^4^, 1^6^, 1^7^) of cells using the Points actor.
-   using a plane to ‘slice’ the same number of cells (using the Scene.slice method).
-   rendering more than 1000 individual meshes representing brain regions from the Allen institute’s mouse brain.
-   making a short (3 s, 10 fps) animation of a spinning brain with several brain regions’ meshes displayed.
-   rendering (10 times) a 3D image representing the voxel-wise expression levels of gene Gpr161 in the mouse brain (data from the Allen Institute).

For each test, we estimated the time necessary to complete the test script as well as the frame rate of the interactive rendering. Four machines were used for benchmark tests (see [Table 1](#table1)). The results of the benchmark tests (see Key resource table) illustrate that although a GPU improves performance, in the absence of a dedicated GPU brainrender can handle rich interactive visualizations (for most user cases, the number of rendered mesh vertices is much lower than that used in the tests).